# Azure OpenAI Service Function Calling 基礎
Azure OpenAI Service では Function Calling（関数呼び出し） の機能が利用可能です。これにより、ユーザーからの質問や指示をプログラムの関数と接続することが容易になります。Function Calling に対応するモデルは、事前に定義された関数に基づいて、インテリジェントに呼び出すべき関数を作成し、データ出力を構造化します。ChatGPT プラグインなどの外部 APIを呼び出して質問に答えるアシスタントを作成することもできます。

https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling?tabs=python

## 事前準備

この Python サンプルを実行するには、以下が必要です：

- Azure OpenAI Service にアクセスできる[承認済み](https://aka.ms/oai/access) Azure サブスクリプション
- Azure OpenAI Service への GPT-3.5 Turbo / GPT-4 モデルのデプロイメント。
- Azure OpenAI Service の接続とモデル情報
  - OpenAI API キー
  - OpenAI GPT-3.5 Turbo / GPT-4 モデルのデプロイメント名
  - OpenAI API バージョン
- Python (この手順はバージョン 3.10.x でテストされています)

これらのデモには、Visual Studio Code と [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) を使用できます。


## パッケージのインストール


In [ ]:
!pip install openai

In [ ]:
import openai
openai.__version__

## 必要なライブラリと環境変数のインポート


## Azure OpenAI の設定
接続情報はセキュリティ面から直接記述するよりも、環境変数や [dotenv](https://pypi.org/project/python-dotenv/) からロードする方法をおすすめします。

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

#os.environ["AZURE_OPENAI_API_KEY"] = "Your OpenAI API Key"
#os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<Your OpenAI Service>.openai.azure.com/"

#これは、モデルをデプロイしたときにデプロイメントに選んだカスタム名に対応します。
#AZURE_OPENAI_DEPLOYMENT_NAME = "gpt-35-turbo"

## 関数呼び出し（非推奨）
この手法での関数呼び出しは `2023-07-01-preview` API バージョンで使用でき、`gpt-35-turbo`、`gpt-35-turbo-16k`、`gpt-4`、`gpt-4-32k` のバージョン `0613` で動作します。

In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [ ]:
messages= [
    {"role": "user", "content": "まず鎌倉幕府第二代征夷大将軍の名前を調べる必要があります"}
]

functions= [
    {
        "name": "PeopleSearchTool",
        "description": "日本の歴史の人物情報の検索に便利です。ユーザーの質問から検索クエリを生成して検索します。クエリは文字列のみを受け付けます",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "検索クエリー"
                }
            },
            "required": [
                "query"
            ]
        }
    },
    {
        "name": "CafeSearchTool",
        "description": "武将にゆかりのあるカフェを検索するのに便利です。カフェの検索クエリには、武将の名前を入力してください。",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "検索クエリー"
                }
            },
            "required": [
                "query"
            ]
        }
    },
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), # model = "deployment_name"
    messages= messages,
    functions = functions,
    function_call="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))

モデルが関数を呼び出す必要があると判断した場合、API からの応答には `function_call` プロパティが含まれます。`function_call` プロパティには、呼び出す関数の名前と、その関数に渡す引数が含まれます。 引数は、関数を呼び出すために解析して使用できる JSON 文字列です。


## 関数呼び出し（移行後）
API の `2023-12-01-preview` バージョンのリリースに伴い、`functions` および `function_call` パラメーターは非推奨になりました。 `functions` に置き換わるのは `tools` パラメーターです。 `function_call` に置き換わるのは `tool_choice` パラメーターです。

In [ ]:
messages= [
    {"role": "user", "content": "まず鎌倉幕府第二代征夷大将軍の名前を調べる必要があります"}
]

tools= [
    {
        "type": "function",
        "function": {
            "name": "PeopleSearchTool",
            "description": "日本の歴史の人物情報の検索に便利です。ユーザーの質問から検索クエリを生成して検索します。クエリは文字列のみを受け付けます",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "検索クエリー"
                    }
                },
                "required": [
                    "query"
                ]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "CafeSearchTool",
            "description": "武将にゆかりのあるカフェを検索するのに便利です。カフェの検索クエリには、武将の名前を入力してください。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "検索クエリー"
                    }
                },
                "required": [
                    "query"
                ]
            }
        }
    }
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), # model = "deployment_name"
    messages= messages,
    tools = tools,
    tool_choice="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))

In [ ]:
messages= [
    {"role": "user", "content": "鎌倉幕府第二代征夷大将軍の名前は源頼家であることがわかりました。次に、源頼家ゆかりの地にあるカフェを調べます。"}
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), # model = "deployment_name"
    messages= messages,
    tools = tools,
    tool_choice="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))


## 並列関数呼び出し
並列関数呼び出しを使用すると、複数の関数呼び出しを同時に実行できるため、並列実行と結果の取得が可能になります。 これにより、必要な API 呼び出しの数が減り、全体的なパフォーマンスが向上します。

### サポートされているモデル
- `gpt-35-turbo (1106)`
- `gpt-4 (1106-preview)`

### サポートされる API バージョン
- `2023-12-01-preview`

サポートされていないモデルを用いると、単一の関数しか返却されません。

In [ ]:
from openai import AzureOpenAI
import os

client_new = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [ ]:
messages= [
    {"role": "user", "content": "サンフランシスコ、東京、パリの天気は？"}
]

tools= [
     {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "指定した場所の現在の天気を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "都市と州（例：San Francisco, CA）",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

response = client_new.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), # model = "deployment_name"
    messages= messages,
    tools = tools,
    tool_choice="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))

たとえば、単純な天気アプリでは、複数の場所の天気を同時に取得したい場合があります。 これにより、`tool_calls` 配列内にそれぞれ一意の `id` を持つ 3 つの関数呼び出しを含むチャット完了メッセージが生成されます。 これらの関数呼び出しに応答する場合は、それぞれ 1 つの関数呼び出しの結果を含む 3 つの新しいメッセージとともに、`tools_calls` から `id` を参照する `tool_call_id` を会話に追加します。

## 実行する関数の定義
モデルは呼び出すべき関数しか返却しません。関数を実行するフェーズでは、ユーザーに代わって世界に影響を与えるアクション (電子メールの送信、オンラインでの投稿、購入など) を実行する前に、ユーザー確認フローを組み込むことを強くおすすめします。

In [ ]:
import json
# 同じ天気を返すようにハードコードされた関数の例
# 本番環境では、これはバックエンドAPIか外部APIとなる
def get_current_weather(location, unit="celsius"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [ ]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
tool_calls

## 関数の実行

In [ ]:
# Step 2: モデルが関数を呼び出したいかどうかをチェックする
if tool_calls:
    # Step 3: 関数を呼び出す
    # Note: JSONレスポンスが常に有効であるとは限りません。
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # この例では関数は1つだけですが、複数の関数を使うこともできます。
    messages.append(response_message)  # アシスタントの返答で messages を拡張
    # Step 4: 各関数の呼び出しとレスポンスの情報をモデルに送信する。
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit")
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # 関数応答で messages を拡張
    #print(messages)
    second_response = client_new.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        messages=messages,
    )  # モデルから新しいレスポンスを取得し、そこで関数のレスポンスを確認する
    #print(second_response)
    print(second_response.choices[0].message.content)